In [1]:
#Importing required packages...
import cv2, os, time
import numpy as np
from keras.models import load_model
from pygame import mixer

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Setting the directory to required path...
os.chdir('C:\\Users\\lenovo\\Downloads\\Drowsiness detection')

In [3]:
#Checking the current directory...
os.getcwd()

'C:\\Users\\lenovo\\Downloads\\Drowsiness detection'

In [4]:
#Initiating the mixer library
mixer.init()
sound= mixer.Sound('alarm.wav')  #Reading the sound file and storing in object

#Creating objects of different cascasde classifiers
face= cv2.CascadeClassifier('Haar cascade files\haarcascade_frontalface_alt.xml')
leye= cv2.CascadeClassifier('Haar cascade files\haarcascade_lefteye_2splits.xml')
reye= cv2.CascadeClassifier('Haar cascade files\haarcascade_righteye_2splits.xml')

In [5]:
lbl=['Close','Open']  #Declaring lables so that we can use later to determine whether the eyes are closed or not...

model= load_model('Model\cnncat2.h5')  #Loading the pre-trained model..
path= os.getcwd()  #Getting the path
cap= cv2.VideoCapture(0)  #Declaring object to open our camera and capture the video.
font= cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [6]:
#Writing code to detect drowsiness...
while(True):
    ret, frame = cap.read()  #Initializing web cam to capture video
    height,width = frame.shape[:2]  #Setting height and width of captured frame.

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Converting the frame to GRAYSCALE as our model understand only Grayscale images.
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))   #Detecting Face
    left_eye = leye.detectMultiScale(gray)  #Detecting Left eye
    right_eye =  reye.detectMultiScale(gray)  #Detecting Right eye

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED ) #Drawing a rectangle upon the entire frame

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1)  #Drawing a rectangle upon face whenever its detected.

    for (x,y,w,h) in right_eye:   #Loop to detect Right_eye
        r_eye=frame[y:y+h,x:x+w]  #Reading Right_eye
        count=count+1   #Initializing count once eye is detect
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)  #Converting detected Right_eye to GrayScale
        r_eye = cv2.resize(r_eye,(24,24))  #Resizing the detected Right_eye
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)  #Reshaping the detected Right_eye     
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict_classes(r_eye) #Predicting the class of the detected Right_eye to determine whether the eye is Closed or Open using loaded pre-trained model.
        if(rpred[0]==1):  #Declaring the detected Right_eye as Open if model predicted as 1
            lbl='Open' 
        if(rpred[0]==0):  #Declaring the detected Right_eye as Closed if model predicted as 0
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:   #Loop to detect Left_eye
        l_eye=frame[y:y+h,x:x+w]  #Reading Left_eye
        count=count+1  #Initializing count once eye is detect
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY) #Converting detected Left_eye to GrayScale 
        l_eye = cv2.resize(l_eye,(24,24))  #Resizing the detected Left_eye
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)  #Reshaping the detected Left_eye
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = model.predict_classes(l_eye) #Predicting the class of the detected Left_eye to determine whether the eye is Closed or Open using loaded pre-trained model.
        if(lpred[0]==1):  #Declaring the detected Left_eye as Open if model predicted as 1
            lbl='Open'   
        if(lpred[0]==0):  #Declaring the detected Left_eye as Closed if model predicted as 0
            lbl='Closed'
        break

    if(rpred[0]==0 and lpred[0]==0): #If both the eyes are Closed then we would be increasing the score
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA) #Showing the text as Closed in the frame
    
    else:    #If both the eyes are Open then we would be decreasing the score
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)  #Showing the text as Open in the frame
    
        
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA) #Showing the Score in realtime in the frame
    if(score>8):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()   #Once we reach a score>8 then we would been a sound.
            
        except:  #isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()